In [1]:
# #############################################################################
# Importing necessary libraries

import numpy as np
import pandas as pd
import requests as requests
import json
from pathlib import Path
from m3inference import M3Inference
from collections import OrderedDict
import pprint
import urllib.request
import random
import re
import json
import os
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import csv 
import sys
import string 
import zipfile as zf
import operator
import sklearn
from sklearn import metrics
from io import BytesIO
from scipy.io import loadmat

In [37]:
### for .mat file for IMDB ###
path_to_meta = "../../../Desktop/Новая папка/GESIS/Gender_Inference/Unpruned_data/IMDB/imdb.mat"
mat = loadmat(path_to_meta)  # load mat-file
mdata = mat['imdb']  # variable in mat file
mdtype = mdata.dtype
ndata = {n: mdata[n][0, 0] for n in mdtype.names}
columns = [n for n, v in ndata.items()]# if v.size == ndata['numIntervals']]

In [38]:
### for .mat file for IMDB ###
dob = mdata['dob'][0,0][0]
photo_taken = mdata['photo_taken'][0,0][0]
full_path = [mdata['full_path'][0,0][0][n][0] for n in range(len(mdata['full_path'][0,0][0]))]
gender = mdata['gender'][0,0][0]
name = [mdata['name'][0,0][0][n][0] for n in range(len(mdata['name'][0,0][0]))]
face_location = mdata['face_location'][0,0][0]
face_score = mdata['face_score'][0,0][0]
second_face_score = mdata['second_face_score'][0,0][0]
celeb_names = [mdata['celeb_names'][0,0][0][n][0] for n in range(len(mdata['celeb_names'][0,0][0]))]
celeb_id = mdata['celeb_id'][0,0][0]

annotations = pd.DataFrame({"dob": dob, "photo_taken":photo_taken, "full_path":full_path, "gender":gender, "name":name, "face_location":face_location, "face_score":face_score, "second_face_score":second_face_score, "celeb_id":celeb_id},
                  index=celeb_id)

In [39]:
df = pd.DataFrame({"dob": dob, "photo_taken":photo_taken, "full_path":full_path, "gender":gender, "name":name, "face_location":face_location, "face_score":face_score, "second_face_score":second_face_score, "celeb_id":celeb_id},
                  index=celeb_id)
                  #columns=columns)
df

,celeb_id,dob,face_location,face_score,full_path,gender,name,photo_taken,second_face_score
6488,6488,693726,"[[1072.926, 161.838, 1214.784, 303.696]]",1.459693,01/nm0000001_rm124825600_1899-5-10_1968.jpg,1.0,Fred Astaire,1968,1.118973
6488,6488,693726,"[[477.184, 100.352, 622.592, 245.76]]",2.543198,01/nm0000001_rm3343756032_1899-5-10_1970.jpg,1.0,Fred Astaire,1970,1.852008
6488,6488,693726,"[[114.96964309, 114.96964309, 451.686572359, 4...",3.455579,01/nm0000001_rm577153792_1899-5-10_1968.jpg,1.0,Fred Astaire,1968,2.985660
6488,6488,693726,"[[622.885505643, 424.217503837, 844.339007674,...",1.872117,01/nm0000001_rm946909184_1899-5-10_1968.jpg,1.0,Fred Astaire,1968,NaN
6488,6488,693726,"[[1013.85900236, 233.882042208, 1201.5861278, ...",1.158766,01/nm0000001_rm980463616_1899-5-10_1968.jpg,1.0,Fred Astaire,1968,NaN
11516,11516,702986,"[[686.676820594, 458.467213729, 1198.10043604,...",3.490563,02/nm0000002_rm1075631616_1924-9-16_1991.jpg,0.0,Lauren Bacall,1991,1.345335
11516,11516,702986,"[[78.2356940656, 52.3351293771, 194.254235164,...",3.479189,02/nm0000002_rm1346607872_1924-9-16_2004.jpg,0.0,Lauren Bacall,2004,NaN
11516,11516,702986,"[[64.3086710375, 64.3086710375, 207.267680872,...",3.870171,02/nm0000002_rm1363385088_1924-9-16_2004.jpg,0.0,Lauren Bacall,2004,NaN
11516,11516,702986,"[[212.767236922, 1055.64418461, 526.798092306,...",3.786499,02/nm0000002_rm1411175936_1924-9-16_1991.jpg,0.0,Lauren Bacall,1991,3.629704
11516,11516,702986,"[[242.176, 49.664, 278.528, 86.016]]",3.472944,02/nm0000002_rm1447271168_1924-9-16_2004.jpg,0.0,Lauren Bacall,2004,2.945133


In [8]:
def extract_files(path, file):# Expanding the zip files
    files = zf.ZipFile(path+file, 'r')
    files.extractall(path+'a')
    files.close()

# Convert all the images in the RGB formate
def preprocess_images(file_name, height, width):
    im = Image.open(file_name)
#     if im.size[0] + im.size[1] < 400:
#         print(f'{file_name} is too small. Skip.')
#         return
    resized_im = im.resize((height, width))
    resized_im.convert('RGB').save(file_name)
        

# Generate random numbers
def randomNumber(Numberlength):
    for x in range(Numberlength):
        number= random.randint(1,10001)
    return number

# Running the M3 inference and getting the gender
def M3_inference(output_path, true_gender):
#     try:
    with open('data.jsonl') as json_file:
        data = json.load(json_file)
    pred = m3.infer(data['images'])                     # see docstring for details
    disc=pprint.pprint(pred)

    with open(output_path, 'w', newline='') as output:  # output file is created
        wr = csv.writer(output,quoting=csv.QUOTE_ALL)
        wr.writerow(['Imagename','G_Gender','M_Gender'])
        for tup in pred.items():
            gender_conf = tup[1]['gender'] 
            gender = max(gender_conf.items(), key=operator.itemgetter(1))
            wr.writerow([tup[0], true_gender, gender[0]])
#     except urllib.error.HTTPError as e:
#         print (e)


# VARIABLES TO CHANGE:
path = '../../../Desktop/Новая папка/GESIS/Gender_Inference/Unpruned_data/Gender_Shade/'
file = 'PPB-2017.zip'
true_gender = 'M'

extract_files(path, file)
m3 = M3Inference()

# Running the imageson M3 and getting the output
output_file = path + file.split('.')[0] + '.csv'
    
files_path = path + file.split('.')[0]+'/'
data = {}
data['images'] = []
for Imagename in os.listdir(files_path):                        # Get the images from directory
#     try:
    if Imagename != 'Thumbs.db' and Imagename != '.DS_Store':
        Image_location=files_path+Imagename # Get the image loacation of each image
        preprocess_images(Image_location, 224, 224)
        data['images'].append({
            "description":"", 
            "id": Imagename,
            "img_path": Image_location, 
            "lang": "en", 
            "name": "", 
            "screen_name": ""
        })
#     except:
#         print("Oops!",sys.exc_info()[0],"occured.")
with open('data.jsonl', 'w') as json_file: # json file for m3 is created  
    json.dump(data, json_file)
M3_inference(output_file, true_gender)


05/20/2020 17:08:40 - INFO - m3inference.m3inference -   Version 1.0.3
05/20/2020 17:08:40 - INFO - m3inference.m3inference -   Running on cpu.
05/20/2020 17:08:40 - INFO - m3inference.m3inference -   Will use full M3 model.
05/20/2020 17:08:41 - INFO - m3inference.m3inference -   Model full_model exists at /Users/user/m3/models/full_model.mdl.
05/20/2020 17:08:41 - INFO - m3inference.utils -   Checking MD5 for model full_model at /Users/user/m3/models/full_model.mdl
05/20/2020 17:08:42 - INFO - m3inference.utils -   MD5s match.
05/20/2020 17:08:42 - INFO - m3inference.m3inference -   Loaded pretrained weight at /Users/user/m3/models/full_model.mdl
05/20/2020 17:08:48 - INFO - m3inference.dataset -   1270 data entries loaded.
Predicting...: 100%|██████████| 80/80 [08:39<00:00,  5.21s/it]


OrderedDict([('0967_5_f_SA.jpg',
              {'age': {'19-29': 0.0209,
                       '30-39': 0.0291,
                       '<=18': 0.0158,
                       '>=40': 0.9341},
               'gender': {'female': 0.9783, 'male': 0.0217},
               'org': {'is-org': 0.0283, 'non-org': 0.9717}}),
             ('0656_6_m_RW.jpg',
              {'age': {'19-29': 0.0045,
                       '30-39': 0.0123,
                       '<=18': 0.0052,
                       '>=40': 0.978},
               'gender': {'female': 0.0072, 'male': 0.9928},
               'org': {'is-org': 0.0083, 'non-org': 0.9917}}),
             ('0947_5_m_SA.jpg',
              {'age': {'19-29': 0.0025,
                       '30-39': 0.0065,
                       '<=18': 0.0041,
                       '>=40': 0.9869},
               'gender': {'female': 0.0119, 'male': 0.9881},
               'org': {'is-org': 0.0248, 'non-org': 0.9752}}),
             ('0970_5_m_SA.jpg',
              {'age'

              {'age': {'19-29': 0.0083,
                       '30-39': 0.013,
                       '<=18': 0.007,
                       '>=40': 0.9717},
               'gender': {'female': 0.97, 'male': 0.03},
               'org': {'is-org': 0.0215, 'non-org': 0.9785}}),
             ('0753_6_m_SE.jpg',
              {'age': {'19-29': 0.0126,
                       '30-39': 0.6047,
                       '<=18': 0.0116,
                       '>=40': 0.3711},
               'gender': {'female': 0.0055, 'male': 0.9945},
               'org': {'is-org': 0.0094, 'non-org': 0.9906}}),
             ('1065_5_f_SA.jpg',
              {'age': {'19-29': 0.3475,
                       '30-39': 0.1588,
                       '<=18': 0.3044,
                       '>=40': 0.1893},
               'gender': {'female': 0.759, 'male': 0.241},
               'org': {'is-org': 0.0109, 'non-org': 0.9891}}),
             ('0156_2_m_FL.jpg',
              {'age': {'19-29': 0.0027,
                    

              {'age': {'19-29': 0.0007,
                       '30-39': 0.0012,
                       '<=18': 0.0006,
                       '>=40': 0.9975},
               'gender': {'female': 0.9937, 'male': 0.0063},
               'org': {'is-org': 0.0175, 'non-org': 0.9825}}),
             ('0475_2_m_SW.jpg',
              {'age': {'19-29': 0.0139,
                       '30-39': 0.0224,
                       '<=18': 0.0211,
                       '>=40': 0.9427},
               'gender': {'female': 0.0159, 'male': 0.9841},
               'org': {'is-org': 0.0131, 'non-org': 0.9869}}),
             ('0608_2_m_SW.jpg',
              {'age': {'19-29': 0.0029,
                       '30-39': 0.0093,
                       '<=18': 0.0052,
                       '>=40': 0.9827},
               'gender': {'female': 0.0073, 'male': 0.9927},
               'org': {'is-org': 0.0271, 'non-org': 0.9729}}),
             ('0534_3_m_SW.jpg',
              {'age': {'19-29': 0.0123,
            

                       '30-39': 0.0183,
                       '<=18': 0.0264,
                       '>=40': 0.9366},
               'gender': {'female': 0.489, 'male': 0.511},
               'org': {'is-org': 0.0289, 'non-org': 0.9711}}),
             ('1038_5_f_SA.jpg',
              {'age': {'19-29': 0.1067,
                       '30-39': 0.1189,
                       '<=18': 0.09,
                       '>=40': 0.6844},
               'gender': {'female': 0.5078, 'male': 0.4922},
               'org': {'is-org': 0.0182, 'non-org': 0.9818}}),
             ('1265_5_m_SA.jpg',
              {'age': {'19-29': 0.0244,
                       '30-39': 0.74,
                       '<=18': 0.0154,
                       '>=40': 0.2202},
               'gender': {'female': 0.0088, 'male': 0.9912},
               'org': {'is-org': 0.009, 'non-org': 0.991}}),
             ('0593_3_m_SW.jpg',
              {'age': {'19-29': 0.0083,
                       '30-39': 0.0504,
                    

...

               'org': {'is-org': 0.0131, 'non-org': 0.9869}}),
             ('1212_5_m_SA.jpg',
              {'age': {'19-29': 0.0528,
                       '30-39': 0.7689,
                       '<=18': 0.0244,
                       '>=40': 0.1538},
               'gender': {'female': 0.0132, 'male': 0.9868},
               'org': {'is-org': 0.004, 'non-org': 0.996}}),
             ('0291_2_f_SW.jpg',
              {'age': {'19-29': 0.0004,
                       '30-39': 0.0007,
                       '<=18': 0.0004,
                       '>=40': 0.9986},
               'gender': {'female': 0.9926, 'male': 0.0074},
               'org': {'is-org': 0.0129, 'non-org': 0.9871}}),
             ('0129_2_m_FL.jpg',
              {'age': {'19-29': 0.0007,
                       '30-39': 0.0013,
                       '<=18': 0.0019,
                       '>=40': 0.9961},
               'gender': {'female': 0.024, 'male': 0.976},
               'org': {'is-org': 0.0182, 'non-org': 0.98

                       '>=40': 0.9944},
               'gender': {'female': 0.0408, 'male': 0.9592},
               'org': {'is-org': 0.0126, 'non-org': 0.9874}}),
             ('0666_5_m_RW.jpg',
              {'age': {'19-29': 0.0091,
                       '30-39': 0.0132,
                       '<=18': 0.0117,
                       '>=40': 0.966},
               'gender': {'female': 0.062, 'male': 0.938},
               'org': {'is-org': 0.0138, 'non-org': 0.9862}}),
             ('0555_3_m_SW.jpg',
              {'age': {'19-29': 0.009,
                       '30-39': 0.9597,
                       '<=18': 0.005,
                       '>=40': 0.0263},
               'gender': {'female': 0.0237, 'male': 0.9763},
               'org': {'is-org': 0.0058, 'non-org': 0.9942}}),
             ('0855_1_f_SA.jpg',
              {'age': {'19-29': 0.0014,
                       '30-39': 0.0025,
                       '<=18': 0.0015,
                       '>=40': 0.9947},
               'g

               'org': {'is-org': 0.028, 'non-org': 0.972}}),
             ('0647_6_f_RW.jpg',
              {'age': {'19-29': 0.1019,
                       '30-39': 0.2725,
                       '<=18': 0.0946,
                       '>=40': 0.531},
               'gender': {'female': 0.3889, 'male': 0.6111},
               'org': {'is-org': 0.0065, 'non-org': 0.9935}}),
             ('0790_6_m_SE.jpg',
              {'age': {'19-29': 0.0008,
                       '30-39': 0.0039,
                       '<=18': 0.0016,
                       '>=40': 0.9937},
               'gender': {'female': 0.0065, 'male': 0.9935},
               'org': {'is-org': 0.0133, 'non-org': 0.9867}}),
             ('0195_3_m_FL.jpg',
              {'age': {'19-29': 0.1478,
                       '30-39': 0.3076,
                       '<=18': 0.1041,
                       '>=40': 0.4405},
               'gender': {'female': 0.0244, 'male': 0.9756},
               'org': {'is-org': 0.0048, 'non-org': 0.9

                       '>=40': 0.9387},
               'gender': {'female': 0.9902, 'male': 0.0098},
               'org': {'is-org': 0.019, 'non-org': 0.981}}),
             ('0375_2_f_SW.jpg',
              {'age': {'19-29': 0.1322,
                       '30-39': 0.0155,
                       '<=18': 0.8324,
                       '>=40': 0.02},
               'gender': {'female': 0.1046, 'male': 0.8954},
               'org': {'is-org': 0.0061, 'non-org': 0.9939}}),
             ('0437_2_m_SW.jpg',
              {'age': {'19-29': 0.0005,
                       '30-39': 0.0013,
                       '<=18': 0.0016,
                       '>=40': 0.9967},
               'gender': {'female': 0.0103, 'male': 0.9897},
               'org': {'is-org': 0.0106, 'non-org': 0.9894}}),
             ('0800_5_m_SE.jpg',
              {'age': {'19-29': 0.0027,
                       '30-39': 0.0104,
                       '<=18': 0.0049,
                       '>=40': 0.9821},
               '

                       '>=40': 0.9964},
               'gender': {'female': 0.0098, 'male': 0.9902},
               'org': {'is-org': 0.0155, 'non-org': 0.9845}}),
             ('0149_2_m_FL.jpg',
              {'age': {'19-29': 0.0131,
                       '30-39': 0.0175,
                       '<=18': 0.0251,
                       '>=40': 0.9443},
               'gender': {'female': 0.0687, 'male': 0.9313},
               'org': {'is-org': 0.0371, 'non-org': 0.9629}}),
             ('0680_5_m_RW.jpg',
              {'age': {'19-29': 0.0006,
                       '30-39': 0.0018,
                       '<=18': 0.0018,
                       '>=40': 0.9959},
               'gender': {'female': 0.0099, 'male': 0.9901},
               'org': {'is-org': 0.0157, 'non-org': 0.9843}}),
             ('0584_2_m_SW.jpg',
              {'age': {'19-29': 0.2197,
                       '30-39': 0.1572,
                       '<=18': 0.3164,
                       '>=40': 0.3067},
            

In [17]:
file_path = '../../../Desktop/Новая папка/GESIS/Gender_Inference/Unpruned_data/Gender_Shade/PPB-2017.csv'
results = pd.read_csv(file_path)
results.head()

,Imagename,G_Gender,M_Gender
0,0967_5_f_SA.jpg,M,female
1,0656_6_m_RW.jpg,M,male
2,0947_5_m_SA.jpg,M,male
3,0970_5_m_SA.jpg,M,male
4,0428_2_m_SW.jpg,M,male


In [18]:
results = pd.merge(results, meta, left_on='Imagename', right_on='filename')
results['gender'] = results['gender'].apply(lambda x: x.lower())
results.head(10)

,Imagename,G_Gender,M_Gender,image_id,filename,gender,fitzpatrick skin type category,bi.fitz,country
0,0967_5_f_SA.jpg,M,female,967,0967_5_f_SA.jpg,female,5,darker,South Africa
1,0656_6_m_RW.jpg,M,male,656,0656_6_m_RW.jpg,male,6,darker,Rwanda
2,0947_5_m_SA.jpg,M,male,947,0947_5_m_SA.jpg,male,5,darker,South Africa
3,0970_5_m_SA.jpg,M,male,970,0970_5_m_SA.jpg,male,5,darker,South Africa
4,0428_2_m_SW.jpg,M,male,428,0428_2_m_SW.jpg,male,2,lighter,Sweden
5,1080_5_m_SA.jpg,M,male,1080,1080_5_m_SA.jpg,male,5,darker,South Africa
6,0641_6_f_RW.jpg,M,female,641,0641_6_f_RW.jpg,female,6,darker,Rwanda
7,0796_6_m_SE.jpg,M,male,796,0796_6_m_SE.jpg,male,6,darker,Senegal
8,0872_2_m_SA.jpg,M,male,872,0872_2_m_SA.jpg,male,2,lighter,South Africa
9,0618_5_f_RW.jpg,M,female,618,0618_5_f_RW.jpg,female,5,darker,Rwanda


In [19]:
# file_f = '../../../Downloads/imdb_female.csv'
# file_m = '../../../Downloads/imdb_male.csv'

# female_all = pd.read_csv(file_f, names=['name', 'G_Gender', 'M_Gender'])
# male_all= pd.read_csv(file_m)
# full_data = pd.concat([female_all, male_all])# 
# True values
y_true = results['gender']
# Predicted values
#full_data['M_Gender'] = full_data['M_Gender'].apply(lambda x: "F" if x == 'female' else "M")
y_pred = results['M_Gender']

In [20]:
# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

#printing the metrics
metrics_dict=metrics.classification_report(y_true, y_pred,output_dict=True)

#precision:
print('Precision:',round(metrics_dict['weighted avg']['precision'],4))
#Recall
print('Recall:',round(metrics_dict['weighted avg']['recall'],4))
#F1-score
print('F1-score:',round(metrics_dict['weighted avg']['f1-score'],4))
#accuracy
print('Accuracy:',round(metrics_dict['accuracy'],4))

[[494  72]
 [  1 703]]
              precision    recall  f1-score   support

      female      0.998     0.873     0.931       566
        male      0.907     0.999     0.951       704

    accuracy                          0.943      1270
   macro avg      0.953     0.936     0.941      1270
weighted avg      0.948     0.943     0.942      1270

Precision: 0.9476
Recall: 0.9425
F1-score: 0.942
Accuracy: 0.9425
